In [46]:
from __future__ import division
import csv
import numpy as np
import matplotlib.pylab as plt
from scipy.signal import resample
import mne
import pandas as pd
import seaborn as sns
from scipy.signal import hilbert
#from utils import circular_histogram
import pycircstat
import os
from datetime import datetime
# Authors: Alexandre Gramfort <alexandre.gramfort@inria.fr>
#          Stanislas Chambon <stan.chambon@gmail.com>
#          Joan Massich <mailsik@gmail.com>
#
# License: BSD Style.

import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import preprocessing

In [47]:
file_dir='D:/IM 680/byteflies_dots/'

In [48]:
patient_id=1351;duration=30;

In [49]:
patient_list=[1351,2510,2939,4301,5852,5995,7257,7838,7488,8898]

In [50]:
def data_reform(patient_id,duration): 
    data = pd.read_csv( os.path.join(file_dir, str(patient_id)+'.csv')) 

    data=data[data['Sleep stage code']!=99]

    data.rename(columns={'Unnamed: 0':'SUBJID'},inplace=True)

    data['SUBJID']=patient_id

    data['Sleep stage code'].values

    sample1=data.iloc[:,3:-1]

    x = sample1.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler(feature_range= (-1,1))
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)

    int(len(sample1)/duration)

    reformed_xdata=[]
    reformed_ydata=[]
    duration=duration*100;
    for i in range(int(len(sample1)/duration)):
        reformed_xdata.append(x_scaled[duration*i:duration*(i+1)])
        reformed_ydata.append(data['Sleep stage code'].values[i*duration])
    return reformed_xdata,reformed_ydata

In [53]:
reformed_xdata=[]
reformed_ydata=[]
for i in patient_list:
    x, y = data_reform(patient_id,duration)
    reformed_xdata += x
    reformed_ydata += y

In [54]:
len(reformed_xdata)

9820

In [55]:
lb = preprocessing.LabelBinarizer()


In [56]:
lb.fit(reformed_ydata)

LabelBinarizer()

In [57]:
lb.classes_

array([0, 1, 2, 3, 4], dtype=int64)

In [58]:
reformed_ydata_binary=lb.transform(reformed_ydata)

In [16]:
np.save('SleepySignals_X.npy', reformed_xdata)    # .npy extension is added if not given

In [17]:
np.save('SleepySignals_y.npy', reformed_ydata)

In [59]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import itertools
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'DejaVu Sans'
import seaborn as sns

# To be able to save images on server
import matplotlib
%matplotlib inline
from matplotlib import pyplot

from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Importing tensorflow
np.random.seed(42)
import tensorflow as tf


from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM , BatchNormalization
from keras.layers.core import Dense, Dropout
from keras.regularizers import L1L2
from sklearn.model_selection import train_test_split

In [99]:
xdata=np.asarray(reformed_xdata)

In [100]:
x_train, x_test, y_train, y_test = train_test_split(xdata, reformed_ydata_binary, test_size=0.2, random_state=42)

In [101]:
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

In [102]:
print(timesteps)
print(input_dim)
print(len(x_train))
print(len(y_train))

3000
16
7856
7856


In [103]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [104]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 165       
Total params: 6,437
Trainable params: 6,437
Non-trainable params: 0
_________________________________________________________________


In [105]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=batch_size,
          validation_data=(x_test, y_test),
          epochs=epochs)

Epoch 1/30
491/491 [==============================] - 491s 995ms/step - loss: 1.0444 - accuracy: 0.6295 - val_loss: 0.8885 - val_accuracy: 0.6904
Epoch 2/30
491/491 [==============================] - 5199s 11s/step - loss: 0.9052 - accuracy: 0.6852 - val_loss: 0.8157 - val_accuracy: 0.6792
Epoch 3/30
491/491 [==============================] - 473s 963ms/step - loss: 0.8521 - accuracy: 0.7021 - val_loss: 0.7397 - val_accuracy: 0.7301
Epoch 4/30
491/491 [==============================] - 6102s 12s/step - loss: 0.8349 - accuracy: 0.7127 - val_loss: 0.8422 - val_accuracy: 0.6782
Epoch 5/30
491/491 [==============================] - 474s 965ms/step - loss: 0.8183 - accuracy: 0.7047 - val_loss: 0.7161 - val_accuracy: 0.7709
Epoch 6/30
 68/491 [===>..........................] - ETA: 15:47:56 - loss: 0.7730 - accuracy: 0.7197